In [ ]:
#pandas根据一列数据更新另一列
def number_to_flag(number):
    if number > 0:
        return '涨'
    elif number == 0:
        return '平'
    else:
        return '跌'

 df['涨跌'] = df['p_change'].map(number_to_flag)


In [ ]:
import numpy as np 
df['涨跌'] = np.sign(df['p_change']) 
df['涨跌'].replace([-1,0,1], ['跌','平','涨'], inplace = True) 